In [1]:
import logging
import helpers.hdbg as hdbg
import helpers.hprint as hprint

import pandas as pd
import helpers.hparquet as hparque

/app/amp/helpers/hparquet.py:19: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-86bd63f1-8f5c-4161-9fa9-6094fdb1a29a.json'


In [3]:
merged_data = pd.read_csv("/shared_data/juraj_CmTask6864/2023_merged-Data.csv", nrows=50000)

In [4]:
merged_data.head()

,#RIC,Alias Underlying RIC,Domain,Date-Time,GMT Offset,Type,Open,High,Low,Last,Volume,No. Trades,Open Bid,High Bid,Low Bid,Close Bid,No. Bids,Open Ask,High Ask,Low Ask,Close Ask,No. Asks,Open Yld,High Yld,Low Yld,Close Yld,No. Ylds,Open Bid Yld,High Bid Yld,Low Bid Yld,Close Bid Yld,No. Bid Ylds,Open Ask Yld,High Ask Yld,Low Ask Yld,Close Ask Yld,No. Ask Ylds,Open Zero Yld,High Zero Yld,Low Zero Yld,Close Zero Yld,No. Zero Ylds,Open Discount Factor,High Discount Factor,Low Discount Factor,Close Discount Factor,No. Discount Factors,Open Bid Size,High Bid Size,Low Bid Size,Close Bid Size,Open Ask Size,High Ask Size,Low Ask Size,Close Ask Size,Open Mid Price,High Mid Price,Low Mid Price,Close Mid Price
0,A,NaN,Market Price,2023-01-03T04:50:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6014,0.6014,0.6014,0.6014,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,NaN,Market Price,2023-01-03T09:00:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,0.0,139.22,0.0,139.22,5.0,154.7,165.0,154.70,154.70,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,1.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN
2,A,NaN,Market Price,2023-01-03T09:15:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,NaN,Market Price,2023-01-03T12:00:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,7.0,NaN,140.0,146.00,140.0,146.00,3.0,154.7,154.7,152.88,152.88,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,7.0,1.0,7.0,NaN,NaN,NaN,NaN
4,A,NaN,Market Price,2023-01-03T12:40:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,146.0,146.00,146.0,146.00,1.0,152.5,152.5,152.50,152.50,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [23]:
def read_csv_and_get_timestamp_summary(file_path, chunksize=50000):
    """
    Read a CSV file in chunks and calculate the minimum and maximum timestamp for each unique value in a specified column.

    Parameters:
    - file_path (str): Path to the CSV file.
    - chunksize (int): Number of rows to read in each chunk.

    Returns:
    - pd.DataFrame: DataFrame containing unique values, min_timestamp, and max_timestamp for each unique value.
    """
    column_name = "#RIC"
    # Initialize an empty DataFrame to store results
    result_df = []

    # Initialize a dictionary to store min and max timestamps for each unique value
    timestamps_dict = {}

    # Use the Pandas read_csv function with chunksize
    chunks = pd.read_csv(file_path, chunksize=chunksize)

    # Process each chunk
    for i, chunk in enumerate(chunks):
        chunk["timestamp"] = pd.to_datetime(chunk["Date-Time"]) - pd.to_timedelta(chunk['GMT Offset'], unit='h')
        # Iterate over unique values in the specified column
        for unique_value in chunk[column_name].unique():
            # Get or initialize min and max timestamps for the current unique value
            min_timestamp, max_timestamp = timestamps_dict.get(unique_value, (pd.Timestamp.max.tz_localize('UTC'), pd.Timestamp.min.tz_localize('UTC')))

            # Update min and max timestamps based on the current chunk
            min_timestamp = min(min_timestamp, chunk.loc[chunk[column_name] == unique_value, 'timestamp'].min())
            max_timestamp = max(max_timestamp, chunk.loc[chunk[column_name] == unique_value, 'timestamp'].max())

            # Store the updated timestamps in the dictionary
            timestamps_dict[unique_value] = (min_timestamp, max_timestamp)
        print(f"Chunk processd {i}")

    # Convert the timestamps dictionary to a DataFrame
    for unique_value, (min_timestamp, max_timestamp) in timestamps_dict.items():
        result_df.append([unique_value, min_timestamp, max_timestamp])

    return result_df

In [24]:
merged_data_summary = read_csv_and_get_timestamp_summary("/shared_data/juraj_CmTask6864/2023_merged-Data.csv")

Chunk processd 0
Chunk processd 1
Chunk processd 2
Chunk processd 3
Chunk processd 4
Chunk processd 5
Chunk processd 6
Chunk processd 7
Chunk processd 8
Chunk processd 9
Chunk processd 10
Chunk processd 11
Chunk processd 12
Chunk processd 13
Chunk processd 14
Chunk processd 15
Chunk processd 16
Chunk processd 17
Chunk processd 18
Chunk processd 19
Chunk processd 20
Chunk processd 21
Chunk processd 22
Chunk processd 23
Chunk processd 24
Chunk processd 25
Chunk processd 26
Chunk processd 27
Chunk processd 28
Chunk processd 29
Chunk processd 30
Chunk processd 31
Chunk processd 32
Chunk processd 33
Chunk processd 34
Chunk processd 35
Chunk processd 36
Chunk processd 37
Chunk processd 38
Chunk processd 39
Chunk processd 40
Chunk processd 41
Chunk processd 42
Chunk processd 43
Chunk processd 44
Chunk processd 45
Chunk processd 46
Chunk processd 47
Chunk processd 48
Chunk processd 49
Chunk processd 50
Chunk processd 51
Chunk processd 52
Chunk processd 53
Chunk processd 54
Chunk processd 55
Ch

Chunk processd 437
Chunk processd 438
Chunk processd 439
Chunk processd 440
Chunk processd 441
Chunk processd 442
Chunk processd 443
Chunk processd 444
Chunk processd 445
Chunk processd 446
Chunk processd 447
Chunk processd 448
Chunk processd 449
Chunk processd 450
Chunk processd 451
Chunk processd 452
Chunk processd 453
Chunk processd 454
Chunk processd 455
Chunk processd 456
Chunk processd 457
Chunk processd 458
Chunk processd 459
Chunk processd 460
Chunk processd 461
Chunk processd 462
Chunk processd 463
Chunk processd 464
Chunk processd 465
Chunk processd 466
Chunk processd 467
Chunk processd 468
Chunk processd 469
Chunk processd 470
Chunk processd 471
Chunk processd 472
Chunk processd 473
Chunk processd 474
Chunk processd 475
Chunk processd 476
Chunk processd 477
Chunk processd 478
Chunk processd 479
Chunk processd 480
Chunk processd 481
Chunk processd 482
Chunk processd 483
Chunk processd 484
Chunk processd 485
Chunk processd 486
Chunk processd 487
Chunk processd 488
Chunk proces

Chunk processd 869
Chunk processd 870
Chunk processd 871
Chunk processd 872
Chunk processd 873
Chunk processd 874
Chunk processd 875
Chunk processd 876
Chunk processd 877
Chunk processd 878
Chunk processd 879
Chunk processd 880
Chunk processd 881
Chunk processd 882
Chunk processd 883
Chunk processd 884
Chunk processd 885
Chunk processd 886
Chunk processd 887
Chunk processd 888
Chunk processd 889
Chunk processd 890
Chunk processd 891
Chunk processd 892
Chunk processd 893
Chunk processd 894
Chunk processd 895
Chunk processd 896
Chunk processd 897
Chunk processd 898
Chunk processd 899
Chunk processd 900
Chunk processd 901
Chunk processd 902
Chunk processd 903
Chunk processd 904
Chunk processd 905
Chunk processd 906
Chunk processd 907
Chunk processd 908
Chunk processd 909
Chunk processd 910
Chunk processd 911
Chunk processd 912
Chunk processd 913
Chunk processd 914
Chunk processd 915
Chunk processd 916
Chunk processd 917
Chunk processd 918
Chunk processd 919
Chunk processd 920
Chunk proces

Chunk processd 1286
Chunk processd 1287
Chunk processd 1288
Chunk processd 1289
Chunk processd 1290
Chunk processd 1291
Chunk processd 1292
Chunk processd 1293
Chunk processd 1294
Chunk processd 1295
Chunk processd 1296
Chunk processd 1297
Chunk processd 1298
Chunk processd 1299
Chunk processd 1300
Chunk processd 1301
Chunk processd 1302
Chunk processd 1303
Chunk processd 1304
Chunk processd 1305
Chunk processd 1306
Chunk processd 1307
Chunk processd 1308
Chunk processd 1309
Chunk processd 1310
Chunk processd 1311
Chunk processd 1312
Chunk processd 1313
Chunk processd 1314
Chunk processd 1315
Chunk processd 1316
Chunk processd 1317
Chunk processd 1318
Chunk processd 1319
Chunk processd 1320
Chunk processd 1321
Chunk processd 1322
Chunk processd 1323
Chunk processd 1324
Chunk processd 1325
Chunk processd 1326
Chunk processd 1327
Chunk processd 1328
Chunk processd 1329
Chunk processd 1330
Chunk processd 1331
Chunk processd 1332
Chunk processd 1333
Chunk processd 1334
Chunk processd 1335


In [47]:
merged_data_summary = pd.DataFrame(merged_data_summary, columns=["currency_pair", "min_timestamp", "max_timestamp"])
merged_data_summary

,currency_pair,min_timestamp,max_timestamp
0,A,2023-01-03 09:50:00+00:00,2023-10-11 03:00:00+00:00
1,AA,2023-01-03 09:50:00+00:00,2023-10-11 03:50:00+00:00
2,AADI.O,2023-01-03 17:00:00+00:00,2023-10-11 02:00:00+00:00
3,AAL.O,2023-01-03 14:00:00+00:00,2023-10-11 03:55:00+00:00
4,AAN,2023-01-03 09:50:00+00:00,2023-10-11 03:00:00+00:00
...,...,...,...
3344,ZVRA.O,2023-03-01 14:00:00+00:00,2023-10-11 01:50:00+00:00
3345,ZWS,2023-01-03 10:30:00+00:00,2023-10-11 03:00:00+00:00
3346,ZYME.O,2023-01-03 14:00:00+00:00,2023-10-11 01:35:00+00:00
3347,ZYXI.O,2023-01-03 14:00:00+00:00,2023-10-11 03:05:00+00:00


In [29]:
estu = pd.read_csv("/shared_data/juraj_CmTask6864/estu_2023.csv.gz", nrows=50000)

In [30]:
estu.head()

,#RIC,Domain,Date-Time,GMT Offset,Type,Open,High,Low,Last,Volume,No. Trades,Open Bid,High Bid,Low Bid,Close Bid,No. Bids,Open Ask,High Ask,Low Ask,Close Ask,No. Asks,Open Yld,High Yld,Low Yld,Close Yld,No. Ylds,Open Bid Yld,High Bid Yld,Low Bid Yld,Close Bid Yld,No. Bid Ylds,Open Ask Yld,High Ask Yld,Low Ask Yld,Close Ask Yld,No. Ask Ylds,Open Zero Yld,High Zero Yld,Low Zero Yld,Close Zero Yld,No. Zero Ylds,Open Discount Factor,High Discount Factor,Low Discount Factor,Close Discount Factor,No. Discount Factors,Open Bid Size,High Bid Size,Low Bid Size,Close Bid Size,Open Ask Size,High Ask Size,Low Ask Size,Close Ask Size,Open Mid Price,High Mid Price,Low Mid Price,Close Mid Price
0,ACET.O,Market Price,2023-01-01T05:00:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACET.O,Market Price,2023-01-01T05:05:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACET.O,Market Price,2023-01-01T05:10:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACET.O,Market Price,2023-01-01T05:15:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACET.O,Market Price,2023-01-01T05:20:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
estu_no_na = estu.dropna(how="all", axis=1)

In [34]:
estu_no_na.head()

,#RIC,Domain,Date-Time,GMT Offset,Type,Open,High,Low,Last,Volume,No. Trades,Open Bid,High Bid,Low Bid,Close Bid,No. Bids,Open Ask,High Ask,Low Ask,Close Ask,No. Asks,Open Bid Size,High Bid Size,Low Bid Size,Close Bid Size,Open Ask Size,High Ask Size,Low Ask Size,Close Ask Size
0,ACET.O,Market Price,2023-01-01T05:00:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACET.O,Market Price,2023-01-01T05:05:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACET.O,Market Price,2023-01-01T05:10:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACET.O,Market Price,2023-01-01T05:15:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACET.O,Market Price,2023-01-01T05:20:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
estu_summary = read_csv_and_get_timestamp_summary("/shared_data/juraj_CmTask6864/estu_2023.csv.gz") 

Chunk processd 0
Chunk processd 1
Chunk processd 2
Chunk processd 3
Chunk processd 4
Chunk processd 5
Chunk processd 6
Chunk processd 7
Chunk processd 8
Chunk processd 9
Chunk processd 10
Chunk processd 11
Chunk processd 12
Chunk processd 13
Chunk processd 14
Chunk processd 15
Chunk processd 16
Chunk processd 17
Chunk processd 18
Chunk processd 19
Chunk processd 20
Chunk processd 21
Chunk processd 22
Chunk processd 23
Chunk processd 24
Chunk processd 25
Chunk processd 26
Chunk processd 27
Chunk processd 28
Chunk processd 29
Chunk processd 30
Chunk processd 31
Chunk processd 32
Chunk processd 33
Chunk processd 34
Chunk processd 35
Chunk processd 36
Chunk processd 37
Chunk processd 38
Chunk processd 39
Chunk processd 40
Chunk processd 41
Chunk processd 42
Chunk processd 43
Chunk processd 44
Chunk processd 45
Chunk processd 46
Chunk processd 47
Chunk processd 48
Chunk processd 49
Chunk processd 50
Chunk processd 51
Chunk processd 52
Chunk processd 53
Chunk processd 54
Chunk processd 55
Ch

In [46]:
estu_summary = pd.DataFrame(estu_summary, columns=["currency_pair", "min_timestamp", "max_timestamp"])
estu_summary

,currency_pair,min_timestamp,max_timestamp
0,ACET.O,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
1,ACI,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
2,ACI.C,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
3,ACTV.K,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
4,ACV.C,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
5,ALTR.O,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
6,AM,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
7,AMN.C,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
8,AMR,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
9,AMRS.O,2023-01-01 10:00:00+00:00,2023-09-23 03:55:00+00:00


In [38]:
backfill = pd.read_csv("/shared_data/juraj_CmTask6864/backfill_2023.csv.gz", nrows=50000)

In [40]:
backfill.head()

,#RIC,Domain,Date-Time,GMT Offset,Type,Open,High,Low,Last,Volume,No. Trades,Open Bid,High Bid,Low Bid,Close Bid,No. Bids,Open Ask,High Ask,Low Ask,Close Ask,No. Asks,Open Yld,High Yld,Low Yld,Close Yld,No. Ylds,Open Bid Yld,High Bid Yld,Low Bid Yld,Close Bid Yld,No. Bid Ylds,Open Ask Yld,High Ask Yld,Low Ask Yld,Close Ask Yld,No. Ask Ylds,Open Zero Yld,High Zero Yld,Low Zero Yld,Close Zero Yld,No. Zero Ylds,Open Discount Factor,High Discount Factor,Low Discount Factor,Close Discount Factor,No. Discount Factors,Open Bid Size,High Bid Size,Low Bid Size,Close Bid Size,Open Ask Size,High Ask Size,Low Ask Size,Close Ask Size,Open Mid Price,High Mid Price,Low Mid Price,Close Mid Price
0,AAIC.K,Market Price,2023-01-01T05:00:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAIC.K,Market Price,2023-01-01T05:05:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAIC.K,Market Price,2023-01-01T05:10:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAIC.K,Market Price,2023-01-01T05:15:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAIC.K,Market Price,2023-01-01T05:20:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
backfill_no_na = backfill.dropna(how="all", axis=1)

In [42]:
backfill_no_na.head()

,#RIC,Domain,Date-Time,GMT Offset,Type,Open,High,Low,Last,Volume,No. Trades,Open Bid,High Bid,Low Bid,Close Bid,No. Bids,Open Ask,High Ask,Low Ask,Close Ask,No. Asks,Open Bid Size,High Bid Size,Low Bid Size,Close Bid Size,Open Ask Size,High Ask Size,Low Ask Size,Close Ask Size
0,AAIC.K,Market Price,2023-01-01T05:00:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAIC.K,Market Price,2023-01-01T05:05:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAIC.K,Market Price,2023-01-01T05:10:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAIC.K,Market Price,2023-01-01T05:15:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAIC.K,Market Price,2023-01-01T05:20:00.000000000Z,-5,Intraday 5Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
backfill_summary = read_csv_and_get_timestamp_summary("/shared_data/juraj_CmTask6864/backfill_2023.csv.gz")

Chunk processd 0
Chunk processd 1
Chunk processd 2
Chunk processd 3
Chunk processd 4
Chunk processd 5
Chunk processd 6
Chunk processd 7
Chunk processd 8
Chunk processd 9
Chunk processd 10
Chunk processd 11
Chunk processd 12
Chunk processd 13
Chunk processd 14
Chunk processd 15
Chunk processd 16
Chunk processd 17
Chunk processd 18
Chunk processd 19
Chunk processd 20
Chunk processd 21
Chunk processd 22
Chunk processd 23
Chunk processd 24
Chunk processd 25
Chunk processd 26
Chunk processd 27
Chunk processd 28
Chunk processd 29
Chunk processd 30
Chunk processd 31
Chunk processd 32
Chunk processd 33
Chunk processd 34
Chunk processd 35
Chunk processd 36
Chunk processd 37
Chunk processd 38
Chunk processd 39
Chunk processd 40
Chunk processd 41
Chunk processd 42
Chunk processd 43
Chunk processd 44
Chunk processd 45
Chunk processd 46
Chunk processd 47
Chunk processd 48
Chunk processd 49
Chunk processd 50
Chunk processd 51
Chunk processd 52
Chunk processd 53
Chunk processd 54
Chunk processd 55
Ch

Chunk processd 437
Chunk processd 438
Chunk processd 439
Chunk processd 440
Chunk processd 441
Chunk processd 442
Chunk processd 443
Chunk processd 444
Chunk processd 445
Chunk processd 446
Chunk processd 447
Chunk processd 448
Chunk processd 449
Chunk processd 450
Chunk processd 451
Chunk processd 452
Chunk processd 453
Chunk processd 454
Chunk processd 455
Chunk processd 456
Chunk processd 457
Chunk processd 458
Chunk processd 459
Chunk processd 460
Chunk processd 461
Chunk processd 462
Chunk processd 463
Chunk processd 464
Chunk processd 465
Chunk processd 466
Chunk processd 467
Chunk processd 468
Chunk processd 469
Chunk processd 470
Chunk processd 471
Chunk processd 472
Chunk processd 473
Chunk processd 474
Chunk processd 475
Chunk processd 476
Chunk processd 477
Chunk processd 478
Chunk processd 479
Chunk processd 480
Chunk processd 481
Chunk processd 482
Chunk processd 483
Chunk processd 484
Chunk processd 485
Chunk processd 486
Chunk processd 487
Chunk processd 488
Chunk proces

In [48]:
backfill_summary = pd.DataFrame(backfill_summary, columns=["currency_pair", "min_timestamp", "max_timestamp"])
backfill_summary

,currency_pair,min_timestamp,max_timestamp
0,AAIC.K,2023-01-01 10:00:00+00:00,2023-12-15 04:55:00+00:00
1,AAMC.K,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00
2,AAOI.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00
3,AAWW.O,2023-01-01 10:00:00+00:00,2023-03-21 03:55:00+00:00
4,ABEO.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00
5,AC,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00
6,ACGN.O,2023-01-18 08:40:00+00:00,2023-09-06 03:55:00+00:00
7,ACOR.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00
8,ACR,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00
9,ACRX.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00


In [50]:
def find_overlap(df1, df2):
    """
    Find overlaps in currency pairs and their corresponding time frames between two DataFrames.

    Parameters:
    - df1 (pd.DataFrame): First DataFrame with columns ['currency_pair', 'min_timestamp', 'max_timestamp'].
    - df2 (pd.DataFrame): Second DataFrame with columns ['currency_pair', 'min_timestamp', 'max_timestamp'].

    Returns:
    - pd.DataFrame: DataFrame containing overlaps in currency pairs and their corresponding time frames.
    """
    # Merge the two DataFrames on the 'currency_pair' column
    merged_df = pd.merge(df1, df2, on='currency_pair', how='inner', suffixes=('_df1', '_df2'))

    # Filter rows where there is an overlap in time frames
    overlap_df = merged_df[(merged_df['max_timestamp_df1'] >= merged_df['min_timestamp_df2']) & (merged_df['min_timestamp_df1'] <= merged_df['max_timestamp_df2'])]

    return overlap_df

In [51]:
find_overlap(backfill_summary, estu_summary)

,currency_pair,min_timestamp_df1,max_timestamp_df1,min_timestamp_df2,max_timestamp_df2
0,ANGI.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
1,BKI,2023-01-01 10:00:00+00:00,2023-09-06 03:55:00+00:00,2023-01-01 10:00:00+00:00,2023-09-06 03:55:00+00:00
2,CHRD.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
3,EDR,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
4,GSM.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
5,MIND.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
6,PAY,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00
7,RXDX.O,2023-01-01 10:00:00+00:00,2023-06-21 03:55:00+00:00,2023-01-01 10:00:00+00:00,2023-06-21 03:55:00+00:00


In [52]:
find_overlap(backfill_summary, merged_data_summary)

,currency_pair,min_timestamp_df1,max_timestamp_df1,min_timestamp_df2,max_timestamp_df2
0,BTU,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 14:00:00+00:00,2023-10-11 03:55:00+00:00
1,CBL,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 10:00:00+00:00,2023-10-11 03:00:00+00:00
2,CHRD.O,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 10:00:00+00:00,2023-10-11 03:55:00+00:00
3,CIVI.K,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 10:00:00+00:00,2023-10-11 03:00:00+00:00
4,CRC,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 10:00:00+00:00,2023-10-11 03:55:00+00:00
5,DEN,2023-01-01 10:00:00+00:00,2023-11-03 03:55:00+00:00,2023-01-03 17:00:00+00:00,2023-10-11 03:55:00+00:00
6,DO,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 14:00:00+00:00,2023-10-11 03:55:00+00:00
7,ENVA.K,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 17:00:00+00:00,2023-10-11 03:00:00+00:00
8,GNK,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 10:05:00+00:00,2023-10-11 03:00:00+00:00
9,NC,2023-01-01 10:00:00+00:00,2023-12-21 05:55:00+00:00,2023-01-03 10:05:00+00:00,2023-10-11 03:00:00+00:00


In [53]:
find_overlap(estu_summary, merged_data_summary)

,currency_pair,min_timestamp_df1,max_timestamp_df1,min_timestamp_df2,max_timestamp_df2
0,ACET.O,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 14:00:00+00:00,2023-10-11 03:15:00+00:00
1,ACI,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 09:50:00+00:00,2023-10-11 03:00:00+00:00
2,ALTR.O,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 14:00:00+00:00,2023-10-11 01:30:00+00:00
3,AM,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 09:55:00+00:00,2023-10-11 03:35:00+00:00
4,AMR,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 09:55:00+00:00,2023-10-11 03:00:00+00:00
5,ARRY.O,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 14:00:00+00:00,2023-10-11 03:55:00+00:00
6,BHVN.K,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 14:00:00+00:00,2023-10-11 03:35:00+00:00
7,CADE.K,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 10:00:00+00:00,2023-10-11 03:00:00+00:00
8,CB,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 10:00:00+00:00,2023-10-11 03:10:00+00:00
9,CHRD.O,2023-01-01 10:00:00+00:00,2024-01-01 09:55:00+00:00,2023-01-03 10:00:00+00:00,2023-10-11 03:55:00+00:00


In [56]:
def get_single_currency_pair_info(file_path, currency_pair):
    column_name = "#RIC"
    # Initialize an empty DataFrame to store results
    result_df = []

    # Use the Pandas read_csv function with chunksize
    chunks = pd.read_csv(file_path, chunksize=50000)

    # Process each chunk
    for i, chunk in enumerate(chunks):
        chunk = chunk.dropna(how="all", axis=1)
        chunk["timestamp"] = pd.to_datetime(chunk["Date-Time"]) - pd.to_timedelta(chunk['GMT Offset'], unit='h')
        chunk = chunk[chunk[column_name] == currency_pair]
        result_df.append(chunk)
        print(f"Chunk processd {i}")
    
    result_df = pd.concat(result_df, ignore_index=True)
    return result_df

In [100]:
backfill_AGNI = get_single_currency_pair_info("/shared_data/juraj_CmTask6864/backfill_2023.csv.gz", "MIND.O")

Chunk processd 0
Chunk processd 1
Chunk processd 2
Chunk processd 3
Chunk processd 4
Chunk processd 5
Chunk processd 6
Chunk processd 7
Chunk processd 8
Chunk processd 9
Chunk processd 10
Chunk processd 11
Chunk processd 12
Chunk processd 13
Chunk processd 14
Chunk processd 15
Chunk processd 16
Chunk processd 17
Chunk processd 18
Chunk processd 19
Chunk processd 20
Chunk processd 21
Chunk processd 22
Chunk processd 23
Chunk processd 24
Chunk processd 25
Chunk processd 26
Chunk processd 27
Chunk processd 28
Chunk processd 29
Chunk processd 30
Chunk processd 31
Chunk processd 32
Chunk processd 33
Chunk processd 34
Chunk processd 35
Chunk processd 36
Chunk processd 37
Chunk processd 38
Chunk processd 39
Chunk processd 40
Chunk processd 41
Chunk processd 42
Chunk processd 43
Chunk processd 44
Chunk processd 45
Chunk processd 46
Chunk processd 47
Chunk processd 48
Chunk processd 49
Chunk processd 50
Chunk processd 51
Chunk processd 52
Chunk processd 53
Chunk processd 54
Chunk processd 55
Ch

Chunk processd 437
Chunk processd 438
Chunk processd 439
Chunk processd 440
Chunk processd 441
Chunk processd 442
Chunk processd 443
Chunk processd 444
Chunk processd 445
Chunk processd 446
Chunk processd 447
Chunk processd 448
Chunk processd 449
Chunk processd 450
Chunk processd 451
Chunk processd 452
Chunk processd 453
Chunk processd 454
Chunk processd 455
Chunk processd 456
Chunk processd 457
Chunk processd 458
Chunk processd 459
Chunk processd 460
Chunk processd 461
Chunk processd 462
Chunk processd 463
Chunk processd 464
Chunk processd 465
Chunk processd 466
Chunk processd 467
Chunk processd 468
Chunk processd 469
Chunk processd 470
Chunk processd 471
Chunk processd 472
Chunk processd 473
Chunk processd 474
Chunk processd 475
Chunk processd 476
Chunk processd 477
Chunk processd 478
Chunk processd 479
Chunk processd 480
Chunk processd 481
Chunk processd 482
Chunk processd 483
Chunk processd 484
Chunk processd 485
Chunk processd 486
Chunk processd 487
Chunk processd 488
Chunk proces

In [101]:
estu_AGNI = get_single_currency_pair_info("/shared_data/juraj_CmTask6864/estu_2023.csv.gz", "MIND.O")

Chunk processd 0
Chunk processd 1
Chunk processd 2
Chunk processd 3
Chunk processd 4
Chunk processd 5
Chunk processd 6
Chunk processd 7
Chunk processd 8
Chunk processd 9
Chunk processd 10
Chunk processd 11
Chunk processd 12
Chunk processd 13
Chunk processd 14
Chunk processd 15
Chunk processd 16
Chunk processd 17
Chunk processd 18
Chunk processd 19
Chunk processd 20
Chunk processd 21
Chunk processd 22
Chunk processd 23
Chunk processd 24
Chunk processd 25
Chunk processd 26
Chunk processd 27
Chunk processd 28
Chunk processd 29
Chunk processd 30
Chunk processd 31
Chunk processd 32
Chunk processd 33
Chunk processd 34
Chunk processd 35
Chunk processd 36
Chunk processd 37
Chunk processd 38
Chunk processd 39
Chunk processd 40
Chunk processd 41
Chunk processd 42
Chunk processd 43
Chunk processd 44
Chunk processd 45
Chunk processd 46
Chunk processd 47
Chunk processd 48
Chunk processd 49
Chunk processd 50
Chunk processd 51
Chunk processd 52
Chunk processd 53
Chunk processd 54
Chunk processd 55
Ch

In [94]:
import numpy as np
def compare_single_asset_data(data1, data2):
    
    def _is_valid_row(row: pd.Series) -> bool:
        for col in cols:
            if np.isnan(row[col + "_A"]) and np.isnan(row[col + "_B"]):
                continue
            if row[col + "_A"] != row[col + "_B"]:
                return False
        return True
    
    data1 = data1.set_index(['timestamp'], drop=True)
    data2 = data2.set_index(['timestamp'], drop=True)
    
    cols = ['Open', 'High', 'Low', 'Last', 'Volume', 'No. Trades', 'Open Bid', 'High Bid', 'Low Bid', 'Close Bid', 'No. Bids', 'Open Ask', 'High Ask', 'Low Ask', 'Close Ask', 'No. Asks', 'Open Bid Size', 'High Bid Size', 'Low Bid Size', 'Close Bid Size', 'Open Ask Size', 'High Ask Size', 'Low Ask Size', 'Close Ask Size']
    
    data1 = data1[cols]
    data2 = data2[cols]
    
    print(f"Size of data1 : {len(data1)}, Size of data2: {len(data2)}")
    merged_df = pd.merge(data1, data2, on=['timestamp'], how='inner', suffixes=('_A', '_B'))
    merged_df["QAcheck"] = merged_df.apply(_is_valid_row, axis=1)
    print(len(merged_df))
    return merged_df
    

In [105]:
comparison = compare_single_asset_data(backfill_AGNI, estu_AGNI)
comparison

Size of data1 : 101904, Size of data2: 105120
101928


,Open_A,High_A,Low_A,Last_A,Volume_A,No. Trades_A,Open Bid_A,High Bid_A,Low Bid_A,Close Bid_A,No. Bids_A,Open Ask_A,High Ask_A,Low Ask_A,Close Ask_A,No. Asks_A,Open Bid Size_A,High Bid Size_A,Low Bid Size_A,Close Bid Size_A,Open Ask Size_A,High Ask Size_A,Low Ask Size_A,Close Ask Size_A,Open_B,High_B,Low_B,Last_B,Volume_B,No. Trades_B,Open Bid_B,High Bid_B,Low Bid_B,Close Bid_B,No. Bids_B,Open Ask_B,High Ask_B,Low Ask_B,Close Ask_B,No. Asks_B,Open Bid Size_B,High Bid Size_B,Low Bid Size_B,Close Bid Size_B,Open Ask Size_B,High Ask Size_B,Low Ask Size_B,Close Ask Size_B,QAcheck
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-01 10:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2023-01-01 10:05:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2023-01-01 10:10:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2023-01-01 10:15:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2023-01-01 10:20:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-21 05:35:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,5.11,5.11,5.11,5.11,0.0,5.98,6.12,5.98,6.12,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.11,5.11,5.11,5.11,0.0,5.98,6.12,5.98,6.12,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True
2023-12-21 05:40:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,5.11,5.11,5.11,5.11,0.0,5.98,6.12,5.98,6.12,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.11,5.11,5.11,5.11,0.0,5.98,6.12,5.98,6.12,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True
2023-12-21 05:45:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,5.11,5.11,5.11,5.11,0.0,5.98,6.12,5.98,6.12,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.11,5.11,5.11,5.11,0.0,5.98,6.12,5.98,6.12,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True


In [103]:
comparison[comparison["QAcheck"] == False]

,Open_A,High_A,Low_A,Last_A,Volume_A,No. Trades_A,Open Bid_A,High Bid_A,Low Bid_A,Close Bid_A,No. Bids_A,Open Ask_A,High Ask_A,Low Ask_A,Close Ask_A,No. Asks_A,Open Bid Size_A,High Bid Size_A,Low Bid Size_A,Close Bid Size_A,Open Ask Size_A,High Ask Size_A,Low Ask Size_A,Close Ask Size_A,Open_B,High_B,Low_B,Last_B,Volume_B,No. Trades_B,Open Bid_B,High Bid_B,Low Bid_B,Close Bid_B,No. Bids_B,Open Ask_B,High Ask_B,Low Ask_B,Close Ask_B,No. Asks_B,Open Bid Size_B,High Bid Size_B,Low Bid Size_B,Close Bid Size_B,Open Ask Size_B,High Ask Size_B,Low Ask Size_B,Close Ask Size_B,QAcheck
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [104]:
backfill_AGNI[["Domain", "Type"]].value_counts()
 

Domain        Type         
Market Price  Intraday 5Min    101904
Name: count, dtype: int64

In [117]:
backfill_assets_set = set(backfill_summary["currency_pair"])
etsu_assets_set = set(estu_summary["currency_pair"])
backfill_etsu_assets = backfill_assets_set.union(etsu_assets_set)
merged_assets_set = set(merged_data_summary["currency_pair"])

only_in_backfill_estu = backfill_etsu_assets.difference(merged_assets_set)

print(f"Total backfill and estu assets : {len(backfill_etsu_assets)}, Total merged : {len(merged_assets_set)}, only_in_backfill_etsu : {len(only_in_backfill_etsu)}") 
                          

Total backfill and estu assets : 560, Total merged : 3349, only_in_backfill_etsu : 499


In [113]:
set(merged_data_summary["currency_pair"]).difference(set(backfill_summary["currency_pair"]).union(set(estu_summary["currency_pair"])))

{'ELD.TO',
 'FMS',
 'NLY',
 'AVAH.O',
 'ENGH.TO',
 'T',
 'TSQ',
 'COLM.O',
 'TM',
 'EQT',
 'RF',
 'MKTX.O',
 'CRGE.O',
 'ESTE.K',
 'ALKT.O',
 'RLJ',
 'DOW',
 'AAN',
 'CMPO.O',
 'EB',
 'NWPX.O',
 'ANDE.O',
 'EQNR.K',
 'ARIS.K',
 'QCRH.O',
 'TRTX.K',
 'CRVL.O',
 'ARDX.O',
 'PII',
 'ABX.TO',
 'AOSL.O',
 'EMN',
 'SCVL.O',
 'WMT',
 'OLMA.O',
 'FOXF.O',
 'LMAT.O',
 'RCL',
 'MPWR.O',
 'TRGP.K',
 'PRU',
 'GEN.O',
 'SPB.TO',
 'ALTI.O',
 'KSS',
 'DV',
 'REXR.K',
 'CBRE.K',
 'HBT.O',
 'PLUG.O',
 'BHR',
 'ATRO.O',
 'MSTR.O',
 'KPTI.O',
 'KRNY.O',
 'GL',
 'CDP',
 'NTCO.K',
 'WTBA.O',
 'AEM',
 'AIRS.O',
 'CACI.K',
 'PHM',
 'AKTS.O',
 'AMBP.K',
 'CENTA.O',
 'GNLX.O',
 'NVRO.K',
 'TRNS.O',
 'BVH',
 'HASI.K',
 'TXN.O',
 'SYK',
 'JPM',
 'NGVT.K',
 'NBIX.O',
 'CNNE.K',
 'VRE',
 'MPB.O',
 'ITIC.O',
 'FN',
 'EQH',
 'JILL.K',
 'LAZR.O',
 'MDRX.O',
 'CLF',
 'INE.TO',
 'WIRE.O',
 'NPK',
 'ZNTL.O',
 'RGNX.O',
 'TBPH.O',
 'NFG',
 'EGY',
 'PHR',
 'ENR',
 'EVER.O',
 'LUN.TO',
 'NNOX.O',
 'CATY.O',
 'HCSG.O',
 'LA

In [110]:
len((set(backfill_summary["currency_pair"]).union(set(estu_summary["currency_pair"]))))

560